# Porting DESeq into python using rpy2#

I will use a small example of [ERCC transcript](https://www.thermofisher.com/order/catalog/product/4456740) from [samples A and B in MAQC data](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3272078/).

In [1]:
import pandas as pd 
import numpy as np

We will read the table and it should only contains count data of ERCC spikeins (rows) and 3 replicates from each of samples A and B (columns).

In [2]:
df = pd.read_table('../test/data/ercc.tsv')
df.head()

,id,A_1,A_2,A_3,B_1,B_2,B_3
0,ERCC-00002,111461,106261,107547,333944,199252,186947
1,ERCC-00003,6735,5387,5265,13937,8584,8596
2,ERCC-00004,17673,13983,15462,5065,3222,3353
3,ERCC-00009,4669,4431,4211,6939,4155,3647
4,ERCC-00012,0,2,0,0,0,0


And here, we will create a design matrix based on the samples in the count table. Note that the sample name has to be used as the ```pd.DataFrame``` index

In [3]:
sample_df = pd.DataFrame({'samplename': df.columns}) \
        .query('samplename != "id"')\
        .assign(sample = lambda d: d.samplename.str.extract('([AB])_', expand=False)) \
        .assign(replicate = lambda d: d.samplename.str.extract('_([123])', expand=False)) 
sample_df.index = sample_df.samplename
sample_df.head()

,samplename,sample,replicate
samplename,,,
A_1,A_1,A,1
A_2,A_2,A,2
A_3,A_3,A,3
B_1,B_1,B,1
B_2,B_2,B,2


Running DESeq2 is jsut like how it is run in ```R```, but instead of the row.name being gene ID for the count table, we can jsut tell the function which column is the gene ID:

In [4]:
from diffexpr.py_deseq import py_DESeq2

dds = py_DESeq2(count_matrix = df,
               design_matrix = sample_df,
               design_formula = '~ replicate + sample',
               gene_column = 'id') # <- telling DESeq2 this should be the gene ID column
    
dds.run_deseq() 
dds.get_deseq_result(contrast = ['sample','B','A'])
res = dds.deseq_result 
res.head()

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing



Using contrast:  [1] "sample" "B"      "A"     



,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,id
ERCC-00002,167917.342729,0.808857,0.047606,16.990537,9.650176e-65,1.102877e-63,ERCC-00002
ERCC-00003,7902.634073,0.521731,0.058878,8.861252,7.912104e-19,4.868987e-18,ERCC-00003
ERCC-00004,10567.048228,-2.330122,0.055754,-41.792764,0.000000e+00,0.000000e+00,ERCC-00004
ERCC-00009,4672.573043,-0.195660,0.061600,-3.176286,1.491736e-03,3.616329e-03,ERCC-00009
ERCC-00012,0.384257,-1.565491,4.047562,-0.386774,6.989237e-01,NaN,ERCC-00012
